## Aggregation Tutorial

In the last section, we inspected the structure of the data and displayed a few example values.

How do we get a deeper feel for the data?  One of the most natural things to do is to create a summary of a large number of values.  For example, you could ask:

 - How many women are in the dataset? How many men?
 - What is the average age?  Youngest age?  Oldest age?
 - What are all the occupations that appear, and how many times does each appear?

We can answer these questions with *aggregation*.  Aggregation combines many values together to create a summary.

To start, we'll aggregate all the values in a table.  (Later, we'll learn how to aggregate over subsets.)

We can do this with the [Table.aggregate](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate) method.

A call to `aggregate` has two parts:

 - The expression to aggregate over (e.g. a field of a `Table`).
 - The *aggregator* to combine the values into the summary.
 
Hail has a large suite of [aggregators](https://hail.is/docs/0.2/aggregators.html) for summarizing data.  Let's see some in action!

### `count`
Aggregators live in the `hl.agg` module.  The simplest aggregator is [count](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count).  It takes no arguments and returns the number of values aggregated.

In [ ]:
import hail as hl
from bokeh.io import output_notebook,show
output_notebook()
hl.init()


hl.utils.get_movie_lens('data/')
users = hl.read_table('data/users.ht')

In [ ]:
users.aggregate(hl.agg.count())

In [ ]:
users.count()

### `stats`
[stats](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats) computes useful statistics about a numeric expression at once.  There are also aggregators for `mean`, `min`, `max`, `sum`, `product` and `array_sum`.

In [ ]:
users.show()

In [ ]:
users.aggregate(hl.agg.stats(users.age))

### `counter`
What about non-numeric data, like the `occupation` field?  

[counter](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter) is modeled on the Python Counter object: it counts the number of times each distinct value occurs in the collection of values being aggregated.

In [ ]:
users.aggregate(hl.agg.counter(users.occupation))

### `filter`
You can filter elements of a collection before aggregation by using [filter](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.filter).

In [ ]:
users.aggregate(hl.agg.filter(users.sex == 'M', hl.agg.count()))

The argument to `filter` should be a Boolean expression.

In [ ]:
users.aggregate(hl.agg.count_where(users.sex == 'M'))

Boolean expressions can be compound, but be sure to use parentheses appropriately. A single '&' denotes logical AND and a single '|' denotes logical OR. 

In [ ]:
users.aggregate(hl.agg.filter((users.occupation == 'writer') | (users.occupation == 'executive'), hl.agg.count()))

In [ ]:
users.aggregate(hl.agg.filter((users.sex == 'F') | (users.occupation == 'executive'), hl.agg.count()))

### `hist`

As we saw in the first tutorial, [hist](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.hist) can be used to build a histogram over numeric data.

In [ ]:
hist = users.aggregate(hl.agg.hist(users.age, 10, 70, 60))
hist

In [ ]:
p = hl.plot.histogram(hist, legend='Age')
show(p)

### `take` and `collect`
There are a few aggregators for collecting values.

- `take` localizes a few values into an array.  It has an optional `ordering`.
- `collect` localizes all values into an array.
- `collect_as_set` localizes all unique values into a set.

In [ ]:
users.aggregate(hl.agg.take(users.occupation, 5))

In [ ]:
users.aggregate(hl.agg.take(users.age, 5, ordering=-users.age))

Warning!  Aggregators like `collect` and `counter` return Python objects and can fail with out of memory errors if you apply them to collections that are too large (e.g. all 50 trillion genotypes in the UK Biobank dataset).